## Backtracking

1.   Permutation
2.   Combination
3.    All Paths





### Permutation

In [0]:
def A_n_k(a, n, k, depth, used, curr, ans):
  '''
  Implement permutation of k items out  of n items
  depth: start from 0, and represent the depth of the search
  used: track what items are  in the partial solution from the set of n
  curr: the current partial solution
  ans: collect all the valide solutions
  '''
  if depth == k: #end condition
    ans.append(curr[::]) # use deepcopy because curr is tracking all partial solution, it eventually become []
    return
  
  for i in range(n):
    if not used[i]:
      # generate the next solution from curr
      curr.append(a[i])
      used[i] = True
      print(curr)
      # move to the next solution
      A_n_k(a, n, k, depth+1, used, curr, ans)
      
      #backtrack to previous partial state
      curr.pop()
      print('backtrack: ', curr)
      used[i] = False
  return

In [4]:
a = [1, 2, 3]
n = len(a)
ans = [[None]]
used = [False] * len(a)
ans = []
A_n_k(a, n, n, 0, used, [], ans)
print(ans)


[1]
[1, 2]
[1, 2, 3]
backtrack:  [1, 2]
backtrack:  [1]
[1, 3]
[1, 3, 2]
backtrack:  [1, 3]
backtrack:  [1]
backtrack:  []
[2]
[2, 1]
[2, 1, 3]
backtrack:  [2, 1]
backtrack:  [2]
[2, 3]
[2, 3, 1]
backtrack:  [2, 3]
backtrack:  [2]
backtrack:  []
[3]
[3, 1]
[3, 1, 2]
backtrack:  [3, 1]
backtrack:  [3]
[3, 2]
[3, 2, 1]
backtrack:  [3, 2]
backtrack:  [3]
backtrack:  []
[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]


### Combination

In [0]:
def C_n_k(a, n, k, start, depth, curr, ans):
  '''
  Implement combination of k items out  of n items
  start: the start of candinate
  depth: start from 0, and represent the depth of the search
  curr: the current partial solution
  ans: collect all the valide solutions
  '''
  if depth == k: #end condition
    ans.append(curr[::]) 
    return
  
  for i in range(start, n):    
    # generate the next solution from curr
    curr.append(a[i])
    # move to the next solution
    C_n_k(a, n, k, i+1, depth+1, curr, ans)

    #backtrack to previous partial state
    curr.pop()
  return

In [0]:
a = [1, 2, 3]
n = len(a)
ans = [[None]]
ans = []
C_n_k(a, n, 2, 0, 0, [], ans)
print(ans)


[[1, 2], [1, 3], [2, 3]]


### All paths

In [0]:
def all_paths(g, s, path, ans):
  '''generate all pahts with backtrack'''
  ans.append(path[::])
  for v in g[s]:
    path.append(v)
    print(path)
    all_paths(g, v, path, ans)
    path.pop()
    print(path, 'backtrack')

In [0]:
al = [[1], [2], [4], [], [3, 5], [6], []]
print(al)

[[1], [2], [4], [], [3, 5], [6], []]


In [0]:
ans = []
path = [0]
all_paths(al, 0, path, ans)
print(ans)

[0, 1]
[0, 1, 2]
[0, 1, 2, 4]
[0, 1, 2, 4, 3]
[0, 1, 2, 4] backtrack
[0, 1, 2, 4, 5]
[0, 1, 2, 4, 5, 6]
[0, 1, 2, 4, 5] backtrack
[0, 1, 2, 4] backtrack
[0, 1, 2] backtrack
[0, 1] backtrack
[0] backtrack
[[0], [0, 1], [0, 1, 2], [0, 1, 2, 4], [0, 1, 2, 4, 3], [0, 1, 2, 4, 5], [0, 1, 2, 4, 5, 6]]


## Constraint Satisfaction Problems with Backtracking and Pruning

First, we build up the board

In [0]:
board = [[5, 3, None, None, 7, None, None, None, None],
         [6, None, None, 1, 9, 5, None, None, None],
         [None, 9, 8, None, None, None, None, 6, None],
         [8, None, None, None, 6, None, None, None, 3], 
         [4, None, None, 8, None, 3, None, None, 1], 
         [7, None, None, None, 2, None, None, None, 6], 
         [None, 6, None, None, None, None, 2, 8, None], 
         [None, None, None, 4, 1, 9, None, None, 5],
         [None, None, None, None, 8, None, None, 7, 9]]

Define how to change the state

In [0]:
def setState(i, j, v, row_state, col_state, grid_state):
  row_state[i] |= 1 << v
  col_state[j] |= 1 << v
  grid_index = (i//3)*3 + (j//3)
  grid_state[grid_index] |= 1 << v
  
def resetState(i, j, v, row_state, col_state, grid_state):
  row_state[i] &= ~(1 << v)
  col_state[j] &= ~(1 << v)
  grid_index = (i//3)*3 + (j//3)
  grid_state[grid_index] &= ~(1 << v)
  
def checkState(i, j, v, row_state, col_state, grid_state):
  row_bit = (1 << v) & row_state[i] != 0
  col_bit = (1 << v) & col_state[j]  != 0
  grid_index = (i//3)*3 + (j//3)
  grid_bit = (1 << v) & grid_state[grid_index]  != 0
  return not row_bit and not col_bit and not grid_bit

Get the empty spots and its values

In [0]:
 def getEmptySpots(board, rows, cols, row_state, col_state, grid_state): 
    ''' get empty spots and find its corresponding values in O(n*n)'''
    empty_spots = {}
    # initialize the state, and get empty spots
    for i in range(rows):
      for j in range(cols):
        if board[i][j]:
            # set that bit to 1
            setState(i, j, board[i][j]-1, row_state, col_state, grid_state)          
        else:
            empty_spots[(i,j)] = []
                    
    # get possible values for each spot
    for i, j in empty_spots.keys():
      for v in range(9):
        if checkState(i, j, v, row_state, col_state, grid_state):
          empty_spots[(i, j)].append(v+1)
          
    return empty_spots

Second, we intialize the state and find empty spots. 

In [0]:
# initialize state
row_state = [0]*9
col_state = [0]*9
grid_state = [0]*9

empty_spots = getEmptySpots(board, 9, 9, row_state, col_state, grid_state)
print(row_state, col_state, grid_state) 
sorted_empty_spots = sorted(empty_spots.items(), key=lambda x: len(x[1]))
print(sorted_empty_spots)

[84, 305, 416, 164, 141, 98, 162, 281, 448] [248, 292, 128, 137, 483, 276, 2, 224, 309] [436, 337, 32, 200, 166, 37, 32, 393, 466]
[((4, 4), [5]), ((6, 5), [7]), ((6, 8), [4]), ((7, 7), [3]), ((0, 3), [2, 6]), ((2, 0), [1, 2]), ((2, 3), [2, 3]), ((2, 4), [3, 4]), ((2, 5), [2, 4]), ((4, 1), [2, 5]), ((5, 1), [1, 5]), ((5, 3), [5, 9]), ((5, 5), [1, 4]), ((6, 4), [3, 5]), ((7, 0), [2, 3]), ((7, 6), [3, 6]), ((8, 5), [2, 6]), ((0, 2), [1, 2, 4]), ((0, 8), [2, 4, 8]), ((1, 1), [2, 4, 7]), ((1, 2), [2, 4, 7]), ((1, 7), [2, 3, 4]), ((2, 8), [2, 4, 7]), ((3, 1), [1, 2, 5]), ((3, 3), [5, 7, 9]), ((3, 5), [1, 4, 7]), ((4, 6), [5, 7, 9]), ((4, 7), [2, 5, 9]), ((5, 7), [4, 5, 9]), ((6, 0), [1, 3, 9]), ((6, 3), [3, 5, 7]), ((7, 1), [2, 7, 8]), ((7, 2), [2, 3, 7]), ((8, 0), [1, 2, 3]), ((0, 5), [2, 4, 6, 8]), ((0, 6), [1, 4, 8, 9]), ((0, 7), [1, 2, 4, 9]), ((1, 6), [3, 4, 7, 8]), ((1, 8), [2, 4, 7, 8]), ((3, 2), [1, 2, 5, 9]), ((3, 6), [4, 5, 7, 9]), ((3, 7), [2, 4, 5, 9]), ((4, 2), [2, 5, 6, 9]), (

Traverse the empty_spots, and fill in. 

In [0]:
def dfs_backtrack(empty_spots, index):
  if index == len(empty_spots):
    return True
  (i, j), vl = empty_spots[index]
  
  for v in vl: #try each value
    # check the state
    if checkState(i, j, v-1, row_state, col_state, grid_state):
      # set the state
      setState(i, j, v-1, row_state, col_state, grid_state)
      # mark the board
      board[i][j] = v
      if dfs_backtrack(empty_spots, index+1):
        return True
      else:
        #backtack to previouse state
        resetState(i, j, v-1, row_state, col_state, grid_state)
        #unmark the board
        board[i][j] = None
  return False
  
  

In [0]:
ans = dfs_backtrack(sorted_empty_spots, 0)
print(ans)
print(board)

True
[[5, 3, 4, 6, 7, 8, 9, 1, 2], [6, 7, 2, 1, 9, 5, 3, 4, 8], [1, 9, 8, 3, 4, 2, 5, 6, 7], [8, 5, 9, 7, 6, 1, 4, 2, 3], [4, 2, 6, 8, 5, 3, 7, 9, 1], [7, 1, 3, 9, 2, 4, 8, 5, 6], [9, 6, 1, 5, 3, 7, 2, 8, 4], [2, 8, 7, 4, 1, 9, 6, 3, 5], [3, 4, 5, 2, 8, 6, 1, 7, 9]]


#### Sudoku Solver

In [0]:
from copy import deepcopy
import time
class SudokoSolver():
  def __init__(self, board):
    self.original_board = deepcopy(board)
    self.board = deepcopy(board)
    self.n = len(board)
    assert (self.n == len(board[0]))
    # initialize state
    self.row_state = [0]*self.n
    self.col_state = [0]*self.n
    self.grid_state = [0]*self.n
    
  def _setState(self, i, j, v):
    self.row_state[i] |= 1 << v
    self.col_state[j] |= 1 << v
    grid_index = (i//3)*3 + (j//3)
    self.grid_state[grid_index] |= 1 << v
  
  def _resetState(self, i, j, v):
    self.row_state[i] &= ~(1 << v)
    self.col_state[j] &= ~(1 << v)
    grid_index = (i//3)*3 + (j//3)
    self.grid_state[grid_index] &= ~(1 << v)
  
  def _checkState(self, i, j, v):
    row_bit = (1 << v) & self.row_state[i] != 0
    col_bit = (1 << v) & self.col_state[j]  != 0
    grid_index = (i//3)*3 + (j//3)
    grid_bit = (1 << v) & self.grid_state[grid_index]  != 0
    return not row_bit and not col_bit and not grid_bit
  
  def reset(self):
    # initialize state
    self.row_state = [0]*self.n
    self.col_state = [0]*self.n
    self.grid_state = [0]*self.n
    self.board = deepcopy(self.original_board)
  
  def _getEmptySpots(self): 
    ''' get empty spots and find its corresponding values in O(n*n)'''
    empty_spots = {}
    # initialize the state, and get empty spots
    for i in range(self.n):
      for j in range(self.n):
        if self.board[i][j]:
            # set that bit to 1
            self._setState(i, j, self.board[i][j]-1)          
        else:
            empty_spots[(i,j)] = []
                    
    # get possible values for each spot
    for i, j in empty_spots.keys():
      for v in range(self.n):
        if self._checkState(i, j, v):
          empty_spots[(i, j)].append(v+1)
          
    return empty_spots
  
  def helper(self, empty_spots, index):
    if index == len(empty_spots):
      return True
    (i, j), vl = empty_spots[index]
  
    for v in vl: #try each value
      # check the state
      if self._checkState(i, j, v-1):
        # set the state
        self._setState(i, j, v-1)
        # mark the board
        self.board[i][j] = v
        if self.helper(empty_spots, index+1):
          return True
        else:
          #backtack to previouse state
          self._resetState(i, j, v-1)
          #unmark the board
          self.board[i][j] = None
    return False
  
  def backtrackSolver(self):
    self.reset()
    empty_spots = self._getEmptySpots()
    empty_spots = [(k, v) for k, v in empty_spots.items() ]
    t0 = time.time()
    ans = self.helper(empty_spots, 0)
    print('total time: ', time.time() - t0)
    return ans
  
  def backtrackSolverSorted(self):
    self.reset()
    empty_spots = self._getEmptySpots()
    empty_spots = sorted(empty_spots.items(), key=lambda x: len(x[1]))
    t0 = time.time()
    ans = self.helper(empty_spots, 0)
    print('sorted total time: ', time.time() - t0)
    return ans

    
    
  
  

In [0]:
board = [[5, 3, None, None, 7, None, None, None, None],
         [6, None, None, 1, 9, 5, None, None, None],
         [None, 9, 8, None, None, None, None, 6, None],
         [8, None, None, None, 6, None, None, None, 3], 
         [4, None, None, 8, None, 3, None, None, 1], 
         [7, None, None, None, 2, None, None, None, 6], 
         [None, 6, None, None, None, None, 2, 8, None], 
         [None, None, None, 4, 1, 9, None, None, 5],
         [None, None, None, None, 8, None, None, 7, 9]]
solver = SudokoSolver(board)
solver.backtrackSolver()
solver.backtrackSolverSorted()

total time:  0.027954578399658203
sorted total time:  0.0004558563232421875


True